In [1]:
import random
import requests
import PyPDF2
import re
import csv
import logging, sys
from SPARQLWrapper import SPARQLWrapper, JSON
logging.basicConfig(filename='reiseberichte.log', encoding='utf-8', level=logging.DEBUG) # Fehlermeldungen etc nach datei loggen

In [2]:
#test-pdfs
protokoll_file = (r"Sitzung Thalmann_26.6.2023_beispiel.pdf") #relative Pfade, bei lokalen notebooks absolute besser
artikel_file = (r"fables.pdf")
philo_file = (r"scotus.pdf")

In [4]:
def read_pdf(file_path): 
    # PDF öffnen und den Text extrahieren
    with open(file_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        text = ''
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

In [5]:
print(read_pdf(protokoll_file)) # Textextraktion testen

Sitzung Thalmann  
Uni Zürich, immer deutschschweiz-romandie, 15 Juni das letzte Mal. Über 20 personen. 
Konzept: Konferenz mit Apéro für Austausch. Veranstaltung ist gratis für die Mitglieder. APéro wird 
von Bibliosuisse beza hlt.  
Stefan Wiederkehr. 
1. Marketing
Citizen Scienc e 10 Personen, vielleicht  
Intervie w mit Stefan Wiederkehr zu CItiz en Science 
4-5 Frag en um die Themati k klar zu machen. Vielleich t an einem Beispiel deutli ch machen.
(Anfang August) 
Lilly fragen wegen Bibliosui sse Intervie w posten 
Anfragen w egen ZB zum posten wegen Interview 
Bibliosuisse info möglich?  
5.7. Mail geht Raus  
Etwas für d ie Werbung überlegen 
Nach dem Anlass ein Berich t schreib en mit Fotos. 
2. Organisatorisch
Vorbereitung für die Tagung  
Apéro von 18-19 Uhr  
10-20 Personen, mit Reserv e au f 25 Personen 
20 Fr. pro Person.  
Eine Woche im Voraus definitiv e Anzahl 
Mit Stefan Wiederkehr Kontakt aufn ehmen, was er braucht. 


In [10]:
def split_words(text):
    # Interpunktion löschen und Wörter trennen, und ein array aller vorkommenden Wörter zurückgeben
#    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'[,.:;?!\(\)\'0-9]+\s', ' ', text) # Interpunktion und Zahlen entfernen, wenn sie von einem whitespace gefolgt werden 
    text = re.sub(r'[,.:;?!\(\)\'0-9]+', '', text) # übrige Interpunktion und Zahlen entfernen
    text = re.sub(r'\n', '', text) # Zeilenumbrüche entfernen, falls einer mitten in einem Wort ist
    all_words = text.split()
    unique_words = set(all_words)
    
#    for word in all_words:
#        if not word in unique_words:
#            unique_words.append(word)
    #words = capitalize_words(words)

    return list(unique_words)

In [11]:
print(len(split_words(read_pdf(protokoll_file))))
print(len(split_words(read_pdf(artikel_file))))
print(len(split_words(read_pdf(philo_file))))

108
464
3103


In [7]:
protokoll_allwords = split_words(read_pdf(protokoll_file))
philo_allwords = split_words(read_pdf(philo_file)) 

print(protokoll_allwords)
print(philo_allwords[0:50])

['Sitzung', 'Thalmann', 'Uni', 'Zürich', 'immer', 'deutschschweiz-romandie', 'Juni', 'das', 'letzte', 'Mal', 'Über', 'personen', 'Konzept', 'Konferenz', 'mit', 'Apéro', 'für', 'Austausch', 'Veranstaltung', 'ist', 'gratis', 'die', 'Mitglieder', 'APéro', 'wird', 'von', 'Bibliosuisse', 'beza', 'hlt', 'Stefan', 'Wiederkehr', 'MarketingCitizen', 'Scienc', 'e', 'Personen', 'vielleicht', 'Intervie', 'w', 'zu', 'CItiz', 'en', 'Science', '-', 'Frag', 'um', 'Themati', 'k', 'klar', 'machen', 'Vielleich', 't', 'an', 'einem', 'Beispiel', 'deutli', 'ch', 'Anfang', 'August', 'Lilly', 'fragen', 'wegen', 'Bibliosui', 'sse', 'posten', 'Anfragen', 'egen', 'ZB', 'zum', 'Interview', 'info', 'möglich', 'Mail', 'geht', 'Raus', 'Etwas', 'd', 'ie', 'Werbung', 'überlegen', 'Nach', 'dem', 'Anlass', 'ein', 'Berich', 'schreib', 'Fotos', 'OrganisatorischVorbereitung', 'Tagung', 'Uhr', 'Reserv', 'au', 'f', 'Fr', 'pro', 'Person', 'Eine', 'Woche', 'im', 'Voraus', 'definitiv', 'Anzahl', 'Mit', 'Kontakt', 'aufn', 'ehmen

In [8]:
#def capitalize_words(word_list):
#    capitalized_words = []
#    for word in word_list:
#        capitalized_words.append(word.capitalize())
#    return capitalized_words

In [1]:
def remove_stopwords(lst, stpwds):
    newlst = lst[:] #In neue Variable kopieren, sodass die alte erhalten bleibt
    for element in stpwds:
        for word in newlst:
            if word == element:
                newlst.remove(word)
    return newlst

In [10]:
    stopwords = ["und", "oder", "der", "die", "das", "von"]

In [17]:
# PDF aufrufen, durchsuchbar machen und bereinigen.
words = remove_stopwords(philo_allwords, stopwords)


In [18]:
print(len(philo_allwords))
print(len(words))

3103
3102


In [13]:
# Test ob Wörter gelöscht wurden.##keine anwendung?
def find_element(lst, element):
    if element in lst:
        index = lst.index(element)
        return index
    else:
        return -1   

In [14]:
def search_and_write(word_list, output_file):
   
        word_column = []
        previous_words_column = []
        next_words_column = []

        for word in word_list:
            if search_gnd(word):
                word_column.append(word)
                if word_list.index(word) == 0:
                    string = "No previous words."
                elif word_list.index(word) == 1:
                    string = "No word here" + " " + word_list[word_list.index(word) - 2] + " " + word_list[word_list.index(word) - 1]
                elif word_list.index(word) == 2:
                    string = "No word here" + " " + "No word here" + " " + word_list[word_list.index(word) - 1]
                else:
                    string = (word_list[word_list.index(word) - 3]) + " " + word_list[word_list.index(word) - 2] + " " + word_list[word_list.index(word) - 1]
                previous_words_column.append(string)  
            
                
                if word_list.index(word) == len(word_list) -1:
                    string = "No next words."
                elif word_list.index(word) == len(word_list) - 2:
                    string = word_list[word_list.index(word) + 1] + " " + "No word here" + " " + "No word here"
                elif word_list.index(word) == len(word_list) - 3:
                    string = word_list[word_list.index(word) + 1] + " " + word_list[word_list.index(word) + 2] + " " + "No word here"
                else:
                    string = (word_list[word_list.index(word) + 1]) + " " + word_list[word_list.index(word) + 2] + " " + word_list[word_list.index(word) + 3]
                next_words_column.append(string) 

        with open(output_file, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)

            # Schreibe den Header
            writer.writerow(['Word', 'Previous Words', 'Next Words'])

            # Schreibe die Daten aus den Listen
            for i in range(len(word_column)):
                writer.writerow([word_column[i], previous_words_column[i], next_words_column[i]])

        print(f'Die Datei {output_file} wurde erfolgreich erstellt.')

In [15]:
#output_file = 'output.csv'
#search_and_write(fewer_words, output_file)

In [16]:
def check_word_in_geonames(word):
    url = f"http://api.geonames.org/searchJSON?name={word}&username=gbmllr" #&fuzzy=0.8 auch möglich

    response = requests.get(url)
        
    if response.status_code == 200:
        geonames_json = response.json()['geonames']
        if not geonames_json == []: # if geonames_json:
            if geonames_json[0]:
                geonames = geonames_json[0]
            else:
                geonames = geonames_json
            if 'countryname' in geonames:   
                print('"' + word + '"' + ' ist ein Ort in' + geonames[0]['countryname'])
            else:
                print('"' + word + '"' + ' ist ein Ort')
            return True
        else:
            print('Keine Treffer für', word)
            return False
    else:
        print('HTML-Status für %s war', word, response.status_code)
        return False 

In [19]:
for word in words[2050:2055]:
    check_word_in_geonames(word)

"Spain" ist ein Ort
Keine Treffer für Summaries
Keine Treffer für Logic
Keine Treffer für “Omne
Keine Treffer für expediens


NameError: name 'geonames' is not defined

In [5]:
def search_gnd(word):
    # GND-API-URL für die Suche nach Geographika
    url = "https://lobid.org/gnd/search?q={}&filter=%2B%28type%3APlaceOrGeographicName%29".format(word)

    # Führe die Anfrage an die GND-API aus
    response = requests.get(url)

    # Überprüfe den Statuscode der Antwort
    if response.status_code == 200:
        # Extrahiere die JSON-Daten aus der Antwort
        data = response.json()

        # Überprüfe, ob es Ergebnisse gibt
        if "member" in data and len(data["member"]) > 0:
            return True
        else:
            return False
    else:
        # Bei einem Fehlercode wird False zurückgegeben
        return False

In [7]:
def has_wikidata_population(word):
    # Erstelle eine SPARQLWrapper-Instanz für die Wikidata-Abfrage-URL
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

    # Setze das SPARQL-Abfrage-Skript mit dem übergebenen Wort
    query = f"""
    PREFIX wd: <http://www.wikidata.org/entity/>
    PREFIX wdt: <http://www.wikidata.org/prop/direct/>

    SELECT ?entity ?population
    WHERE {{
      ?entity wdt:P1082 ?population ;
              rdfs:label "{word}"@en .
    }}
    """

    # Setze das Abfrage-Skript in die SPARQLWrapper-Instanz
    sparql.setQuery(query)

    # Setze das Ausgabeformat auf JSON
    sparql.setReturnFormat(JSON)

    # Führe die SPARQL-Abfrage aus und erhalte die Ergebnisse
    results = sparql.query().convert()

    # Überprüfe, ob es Ergebnisse gibt
    if len(results["results"]["bindings"]) > 0:
        return True
    else:
        return False